# Market Basket Analysis
Market Basket Analysis (MBA) adalah metode analitik populer dalam data mining yang bertujuan mengidentifikasi pola pembelian pelanggan berdasarkan data transaksi. Teknik ini sangat berguna dalam konteks ritel dan e-commerce untuk memahami kecenderungan asosiasi antar produk, sehingga dapat dimanfaatkan untuk strategi promosi, penataan rak, dan rekomendasi produk.

# Impor packages

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
import os
os.getcwd()

'C:\\Users\\LENOVO\\Python\\Intermediate'

# Impor data dari CSV ke DataFrame

In [3]:
df = pd.read_csv('C:/Users/LENOVO/Python/Online Retail Data.csv', header=0)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


# Data cleansing

In [5]:
df_clean = df.copy()

## membuat kolom date

In [6]:
df_clean['order_date'] = pd.to_datetime(df_clean['order_date'])
df_clean = df_clean.rename(columns={'order_date':'date'})

## menghapus semua baris tanpa customer_id

In [7]:
df_clean = df_clean[~df_clean['customer_id'].isna()]

## mengkonversi customer_id menjadi string

In [8]:
df_clean['customer_id'] = df_clean['customer_id'].astype(str)

## menghapus semua baris tanpa product_name

In [9]:
df_clean = df_clean[~df_clean['product_name'].isna()]

## membuat semua product_name berhuruf kecil

In [10]:
df_clean['product_name'] = df_clean['product_name'].str.lower()

## menghapus semua baris dengan product_code atau product_name test

In [11]:
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]

## menghapus baris dengan status cancelled, yaitu yang order_id-nya diawali 'C'

In [12]:
df_clean = df_clean[df_clean['order_id'].str[:1]!='C']

## mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled

In [13]:
df_clean['quantity'] = df_clean['quantity'].abs()

## menghapus baris dengan price bernilai negatif

In [14]:
df_clean = df_clean[df_clean['price']>0]

## membuat nilai amount, yaitu perkalian antara quantity dan price

In [15]:
df_clean['amount'] = df_clean['quantity'] * df_clean['price']

## Menghitung jumlah order per kombinasi product_code dan product_name

In [16]:
most_freq_product_name = df_clean.groupby(
    ['product_code','product_name'], as_index=False).agg(
    order_cnt=('order_id','nunique')).sort_values(
    ['product_code','order_cnt'], ascending=[True,False])

## Memberi peringkat berdasarkan jumlah order dalam tiap product_code

In [17]:
most_freq_product_name['rank'] = most_freq_product_name.groupby(
    'product_code')['order_cnt'].rank(method='first', ascending=False)

## Ambil hanya nama produk dengan peringkat 1 (terbanyak order) untuk tiap product_code

In [18]:
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])

## Gabungkan kembali ke data utama berdasarkan product_code

In [19]:
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')

## Gantikan product_name lama dengan most_freq_product_name

In [20]:
df_clean['product_name'] = df_clean['most_freq_product_name']

## Hapus kolom sementara

In [21]:
df_clean = df_clean.drop(columns='most_freq_product_name')

## menghapus outlier

In [22]:
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

,order_id,product_code,product_name,quantity,date,price,customer_id,amount
0,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,91.80
1,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,51.00
2,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,5.10
3,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,8.50
4,493414,21527,red retrospot traditional teapot,12,2010-01-04 10:28:00,6.95,14590.0,83.40
...,...,...,...,...,...,...,...,...
350087,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,1.25
350088,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69
350089,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69
350090,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,9.90


In [23]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350092 entries, 0 to 350091
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      350092 non-null  object        
 1   product_code  350092 non-null  object        
 2   product_name  350092 non-null  object        
 3   quantity      350092 non-null  int64         
 4   date          350092 non-null  datetime64[ns]
 5   price         350092 non-null  float64       
 6   customer_id   350092 non-null  object        
 7   amount        350092 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 21.4+ MB


# Menyiapkan data basket

## Buat DataFrame basket

In [24]:
basket = pd.pivot_table(df_clean, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
basket

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16272 entries, 493414 to 539988
Columns: 3842 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: int64(3842)
memory usage: 477.1+ MB


## Encode DataFrame basket dengan nilai True untuk semua nilai di atas 0 dan False untuk semua nilai 0

In [26]:
def encode(x):
    if x==0:
        return False
    if x>0:
        return True

basket_encode = basket.applymap(encode)
basket_encode

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15972\3017338639.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encode = basket.applymap(encode)


product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539985,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
basket_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16272 entries, 493414 to 539988
Columns: 3842 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3842)
memory usage: 59.7+ MB


## Ambil transaksi dengan banyaknya produk unik lebih dari 1 saja

In [28]:
basket_filter = basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
basket_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15004 entries, 493414 to 539988
Columns: 3842 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3842)
memory usage: 55.1+ MB


# Mengaplikasikan apriori algorithm

## Buat list frequent itemset (kumpulan produk yang sering dibeli)

In [30]:
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(basket_filter, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

C:\Users\LENOVO\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\LENOVO\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Constant.__init__ got an unexpected keyword argument 's'. Support for arbitrary keyword arguments is deprecated and will be removed in Python 3.15.
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Constant.__init__ missing 1 required positional argument: 'value'. This will become an error in Python 3.15.

MemoryError: Unable to allocate 6.53 GiB for an array with shape (233586, 2, 15004) and data type bool

### Karena error, berikut beberapa alternatifnya

#### Batasi jumlah kolom produk (feature selection)
Kita bisa ambil hanya produk yang sering muncul saja, misalnya yang muncul di minimal 1% transaksi.

In [31]:
from mlxtend.frequent_patterns import apriori

# Hitung total kemunculan tiap produk
frequent_products = basket_filter.sum(axis=0)

# Ambil hanya produk dengan minimal 1% transaksi
min_threshold = 0.01 * len(basket_filter)
selected_products = frequent_products[frequent_products >= min_threshold].index

# Filter kolom basket
basket_filter_small = basket_filter[selected_products]

# Menjalankan algoritma apriori
frequent_itemset = apriori(basket_filter_small, min_support=0.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))

C:\Users\LENOVO\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Constant.__init__ got an unexpected keyword argument 's'. Support for arbitrary keyword arguments is deprecated and will be removed in Python 3.15.
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Constant.__init__ missing 1 required positional argument: 'value'. This will become an error in Python 3.15.
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant i

MemoryError: Unable to allocate 6.53 GiB for an array with shape (233586, 2, 15004) and data type bool

#### Gunakan max_len untuk membatasi kombinasi item
Karena kombinasi semua item sangat banyak, kita bisa batasi maksimal panjang kombinasi item.\
Artinya: hanya cari kombinasi item 1 dan 2, bukan 3, 4, dst.

In [32]:
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(basket_filter_small, min_support=0.01, use_colnames=True, max_len=2)

C:\Users\LENOVO\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Constant.__init__ got an unexpected keyword argument 's'. Support for arbitrary keyword arguments is deprecated and will be removed in Python 3.15.
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\ast.py:602: DeprecationWarning: Constant.__init__ missing 1 required positional argument: 'value'. This will become an error in Python 3.15.
  return Constant(*args, **kwargs)
C:\Users\LENOVO\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant i

MemoryError: Unable to allocate 6.53 GiB for an array with shape (233586, 2, 15004) and data type bool

#### Gunakan algoritma alternatif: FP-Growth
apriori() sangat boros memori karena eksplisit menghitung kombinasi. Jika tetap ingin performa lebih baik, gunakan:

In [33]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemset = fpgrowth(basket_filter_small, min_support=0.01, use_colnames=True)

In [34]:
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))

## Hitung nilai support, confidence, dan lift dari setiap pasangan produk yang mungkin

In [35]:
from mlxtend.frequent_patterns import association_rules

product_association = association_rules(frequent_itemset, metric='confidence', min_threshold=.7).sort_values(
    ['support','confidence'], ascending=[False,False]).reset_index(drop=True)
product_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(red hanging heart t-light holder),(white hanging heart t-light holder),0.058784,0.177953,0.042455,0.722222,4.058510,1.0,0.031995,2.959371,0.800671,0.218525,0.662090,0.480400
1,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.049254,0.074980,0.037457,0.760487,10.142533,1.0,0.033764,3.862089,0.948103,0.431644,0.741073,0.630021
2,(toilet metal sign),(bathroom metal sign),0.026993,0.040589,0.021728,0.804938,19.831353,1.0,0.020632,4.918499,0.975918,0.473837,0.796686,0.670121
3,(red retrospot sugar jam bowl),(red retrospot small milk jug),0.023660,0.037123,0.016796,0.709859,19.121592,1.0,0.015917,3.318652,0.970669,0.381818,0.698673,0.581141
4,(painted metal pears assorted),(assorted colour bird ornament),0.021927,0.078512,0.016596,0.756839,9.639738,1.0,0.014874,3.789618,0.916356,0.197933,0.736121,0.484107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,"(green 3 piece mini dots cutlery set, blue 3 p...",(pink 3 piece mini dots cutlery set),0.011464,0.029992,0.010064,0.877907,29.271370,1.0,0.009720,7.944827,0.977037,0.320594,0.874132,0.606731
82,"(green 3 piece mini dots cutlery set, pink 3 p...",(blue 3 piece mini dots cutlery set),0.011997,0.030525,0.010064,0.838889,27.481853,1.0,0.009698,6.017430,0.975313,0.310062,0.833816,0.584292
83,"(key fob , back door , key fob , front door )","(key fob , shed)",0.012463,0.025060,0.010064,0.807487,32.222153,1.0,0.009752,5.064272,0.981194,0.366505,0.802538,0.604541
84,"(poppy's playhouse kitchen, poppy's playhouse ...",(poppy's playhouse bathroom),0.013730,0.010997,0.010064,0.733010,66.655016,1.0,0.009913,3.704266,0.998709,0.686364,0.730041,0.824081


### Interpretasi Baris 0
* Jika seseorang membeli red hanging heart t-light holder, maka ada peluang 72% dia juga membeli white hanging heart t-light holder.
* Nilai lift = 4.05 artinya pembelian white meningkat 4x lipat lebih besar dibandingkan kalau pembelian tersebut terjadi secara acak (tidak tergantung produk lainnya).
* Artinya, kombinasi ini sangat kuat dan saling mendukung (komplementer).

### Interpretasi Baris 1
* Pelanggan yang membeli sweetheart ceramic memiliki kemungkinan 76% untuk juga membeli strawberry ceramic.
* Lift = 10.14 sangat tinggi → artinya asosiasi ini sangat kuat dan bisa dimanfaatkan sebagai rekomendasi "produk pasangan" (pair product).
* Support 3.7% mungkin terlihat kecil, tapi di e-commerce, ini sudah cukup signifikan karena item unik bisa punya support rendah namun sangat meaningful.

### Interpretasi Baris 2
* Peluang membeli bathroom metal sign jika telah membeli toilet metal sign adalah 80%, dengan lift hampir 20x dari ekspektasi acak → sangat kuat dan terhubung secara logis (produk serupa/tematik).
* Ini cocok untuk strategi rekomendasi yang mendukung dekorasi rumah atau kategori serupa.

### Kesimpulan singkat:
* Banyak aturan asosiasi di output memiliki confidence tinggi (>70%) dan lift >1, bahkan >10 → ini menunjukkan asosiasi yang sangat kuat dan bisa diandalkan.
* Dapat dipilih aturan dengan confidence > 0.8 dan lift > 3 sebagai strategi rekomendasi produk atau bundling penjualan.
* Produk-produk ini kemungkinan besar dibeli bersamaan karena mereka saling melengkapi (warna, fungsi, atau kategori serupa).

| Metrik             | Makna                                                                                                            |
| ------------------ | ---------------------------------------------------------------------------------------------------------------- |
| **support**        | Persentase transaksi yang mengandung kombinasi produk (antecedent + consequent)                                  |
| **confidence**     | Probabilitas seseorang membeli *consequent* jika sudah membeli *antecedent*                                      |
| **lift**           | Seberapa besar pengaruh *antecedent* terhadap *consequent* dibandingkan kejadian acak (lift > 1 artinya positif) |
| **leverage**       | Seberapa besar perbedaan antara support aktual dan support yang diharapkan jika tidak tergantung                 |
| **conviction**     | Ukuran seberapa "penting" hubungan ini; lebih tinggi → lebih dapat dipercaya                                     |
| **zhangs\_metric** | Metode alternatif untuk mengukur asosiasi; mempertimbangkan support dan confidence                               |
| **jaccard**        | Ukuran kemiripan antara antecedent dan consequent                                                                |
| **certainty**      | Seberapa pasti suatu aturan itu benar jika antecedent terpenuhi                                                  |
| **kulczynski**     | Rata-rata simetri dari confidence (A⇒B dan B⇒A); semakin tinggi semakin simetris                                 |
